### 필요한 라이브러리 import

In [28]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import sys
import urllib.request
import urllib.parse
import json
import requests

#selenium의 webdriver를 사용
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
import numpy as np
import faiss
from sklearn.metrics.pairwise import cosine_similarity

### OPENAI, kakao rest api, naver api 키를 .env 파일로부터 load

In [ ]:
# env 파일 로드
load_dotenv()


# openai api, kakao rest api, naver api 키 로드
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
KAKAO_REST_API_KEY = os.getenv('KAKAO_RESTAPI_KEY')
NAVER_API_CLIENT_ID = os.getenv('NAVER_API_CLIENT_ID')
NAVER_API_CLIENT_SECRET = os.getenv('NAVER_API_CLIENT_SECRET')

# OpenAI client 정의
client = OpenAI(api_key = OPENAI_API_KEY)


system_prompt = system_prompt = """반드시 다음 규칙을 지켜 답변하시오:
1. 제공된 문맥에만 기반하여 답변
2. 문맥에 없는 정보는 '관련 정보를 찾을 수 없습니다'로 통일
3. 긍정, 부정적인지를 합 100%으로 각각 답변"""

### OpenAI api를 활용해 ChatGPT 활용하기

In [30]:
def openai_message(message):
    context_chunks = search(query)
    context = "\n\n".join([chunk["text"] for chunk in context_chunks])
    
    completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        { "role": "system", "content": system_prompt },
        { "role": "user", "content": f"문맥:\n{context}\n\n출처: {url}\n\n질문: {message}" }
    ],
    temperature = 0.3,
    max_tokens = 300 # 출력 길이 제한
    )

    print(completion.choices[0].message.content)

In [31]:
def kakao_rest_api(longitude, latitude):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'
    headers = {'Authorization': f'KakaoAK {KAKAO_REST_API_KEY}'}
    params = {'x': longitude, 'y': latitude}
    
    response = requests.get(url, headers=headers, params=params)
    
    # 데이터를 성공적으로 불러온 경우, code = 200
    if response.status_code == 200:
        return response.json()
    else:
        return None

### 카카오 Rest api를 활용해 해당 좌표의 카테고리별 결과 불러오기

In [32]:
def search_by_category(longitude, latitude, category_code):
    url = 'https://dapi.kakao.com/v2/local/search/category.json'
    headers = {'Authorization': f'KakaoAK {KAKAO_REST_API_KEY}'}
    params = {
        'category_group_code': category_code,
        'x': longitude,
        'y': latitude,
        'radius': 1000,  # 검색 반경 설정 (단위: 미터)
        'size': 15       # 한 페이지에 보여질 결과 개수
    }

    # API 요청
    response = requests.get(url, headers=headers, params=params)

    # 응답 처리
    if response.status_code == 200:
        return response.json()  # 성공적으로 데이터를 반환
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None  # 실패 시 None 반환

### 네이버 API를 통한 키워드 검색

In [33]:
def search_naver_blog(keyword):
    client_id = NAVER_API_CLIENT_ID
    client_secret = NAVER_API_CLIENT_SECRET
    # 블로그에 검색할 키워드
    encText = urllib.parse.quote(keyword.encode(encoding = 'UTF-8'))
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText
    
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    # 정상적으로 블로그글이 로드된 경우
    if(rescode==200):
        response_body = response.read()
        return json.loads(response_body.decode('utf-8'))
    # 오류 발생의 경우
    else:
        print("Error Code:" + rescode)

### 블로그 링크 크롤링

In [34]:
def get_naver_blog_content(url):
    # 본문 내용을 저장할 리스트
    contents = []
    
    # Selenium WebDriver 설정하기
    driver = webdriver.Chrome()

    try:
        driver.get(url)
        # iframe 로딩 대기 및 전환
        WebDriverWait(driver, 10).until(
            EC.frame_to_be_available_and_switch_to_it((By.ID, "mainFrame"))
        )

        #본문 내용 크롤링하기
        try:
            # 최신 네이버 블로그
            a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
            contents.append(a)
            
        except NoSuchElementException:
            # 구 버전 네이버 블로그
            a = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
            contents.append(a)
        # print('본문: \n', a)
        
    except Exception as e:
        print(f"오류 발생: {e}")
        
    finally:
        driver.quit() #창닫기
        print("<<본문 크롤링이 완료되었습니다.>>")

    return contents

In [35]:
# 2. 데이터 정리
def clean_text(text):
    # 입력값이 문자열인지 확인
    if not isinstance(text, str):
        text = str(text)  # 문자열로 변환
    text = re.sub(r"[^\w\s가-힣]", " ", text)  # 특수문자 및 이모지 제거
    text = re.sub(r"\s+", " ", text).strip()  # 공백 정리
    return text

In [36]:
# 3. 청크 분할
def chunk_text(text, chunk_size=300, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i+chunk_size])
        if len(chunk) > 10:  # 10자 미만 청크 제외
            chunks.append(chunk)
    return chunks if chunks else ["[No valid chunks]"]  # 빈 청크 방지

In [37]:
# 4. 임베딩 생성
def get_embedding(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

In [38]:
def search(query, top_k=3):
    if not metadata_store:
        return []
    
    # 쿼리 임베딩 생성
    query_embedding = get_embedding(query)
    
    # FAISS 검색 (k 필수 지정)
    distances, indices = index.search(
        np.array([query_embedding]).astype('float32'),  # 2D 배열로 변환
        top_k  # 상위 k개 결과 요청
    )
    
    # 유효한 인덱스 필터링
    valid_indices = [i for i in indices[0] if i < len(metadata_store)]
    return [metadata_store[i] for i in valid_indices]

In [39]:
# 5. 벡터 DB 초기화
embedding_dim = 1536  # text-embedding-3-small 차원
index = faiss.IndexFlatL2(embedding_dim)
metadata_store = []

### main 함수 실행 part

In [44]:
# 좌표 기준 카테고리 검색 rest api 실행. 기준-강원대 자연대학 기준
category_result = search_by_category(127.743288, 37.872316, "FD6")
print(category_result)

{'documents': [{'address_name': '강원특별자치도 춘천시 효자동 618-4', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 중식', 'distance': '265', 'id': '1362999570', 'phone': '0507-1415-2733', 'place_name': '육림객잔', 'place_url': 'http://place.map.kakao.com/1362999570', 'road_address_name': '강원특별자치도 춘천시 서부대성로 207', 'x': '127.74155390732444', 'y': '37.87427198463118'}, {'address_name': '강원특별자치도 춘천시 효자동 174-59', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 한식 > 육류,고기 > 닭요리', 'distance': '738', 'id': '18879265', 'phone': '033-243-2888', 'place_name': '진미닭갈비 본점', 'place_url': 'http://place.map.kakao.com/18879265', 'road_address_name': '강원특별자치도 춘천시 백령로 51', 'x': '127.736742118901', 'y': '37.8681542435598'}, {'address_name': '강원특별자치도 춘천시 효자동 633-11', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 양식 > 피자', 'distance': '272', 'id': '2135313009', 'phone': '033-911-9023', 'place_name': '브릭스피자', 'place_u

In [45]:
for document in category_result["documents"]:
    place_name = document["place_name"]
    print(place_name)
    print(search_naver_blog(place_name))
    print() #머쓱

육림객잔
{'lastBuildDate': 'Tue, 01 Apr 2025 20:56:44 +0900', 'total': 456, 'start': 1, 'display': 10, 'items': [{'title': '(내돈내산)춘천 강대후문 갈비짬뽕 맛집 효자동 <b>육림객잔</b>', 'link': 'https://blog.naver.com/k2jongin/223560846137', 'description': '사장님께서 <b>육림객잔</b>의 짜장밥은 밥 위에 짜장을 부어놓은 비주얼이 아닌, 짜장과 밥을 비벼놓은 모습이라고 하셔서 뭔가 상당히 독특하고 생각되어 이건 꼭 주문해야겠다고 결심했음 ㅎㅎ <b>육림객잔</b>은 김치가... ', 'bloggername': '크리스 코리아 예스춘천', 'bloggerlink': 'blog.naver.com/k2jongin', 'postdate': '20240826'}, {'title': '강원대맛집 <b>육림객잔</b> 퓨전중식당 춘천맛집 갈비짬뽕 추천', 'link': 'https://blog.naver.com/mistuki07/223159897071', 'description': 'ㅋㅋㅋㅋ 강대후문에 <b>육림객잔</b>이라는 퓨전중식당 맛집이 생겼다고 해서 옥토로랑 쉬는 날 아침에... 강원대맛집 <b>육림객잔</b> 분위기 최고다 진짜!!!! 귀엽고 아기자기한 소품들도 볼 수 있어서 좋았고, 벽에... ', 'bloggername': '서쪽의 웨스트에서 불어오는 바람의 윈드', 'bloggerlink': 'blog.naver.com/mistuki07', 'postdate': '20230721'}, {'title': '춘천 강원대맛집 [<b>육림객잔</b>] 국물맛이 깊은 갈비짬뽕 퓨전중식당', 'link': 'https://blog.naver.com/hallo--ween/223184620636', 'description': '강원대맛집 <b>육림객잔</b>. 육림고개에 있을 때부터 꼭 한번 방문하고 싶은 곳이었

In [ ]:
for document in category_result["documents"]:
    address_name = document["address_name"]
    place_name = document["place_name"]

    # 해당 장소의 x, y좌표
    place_x = document["x"]
    place_y = document["y"]
    
    # API 호출 및 데이터 처리
    search_result = search_naver_blog(place_name)
    print(place_name)
    
    # 전처리된 데이터
    cleaned_contents = []
    
    if 'items' in search_result:
        # 네이버 API로 주어진 키워드에 맞는 블로그 링크를 가져오기
        for item in search_result['items']:
            url = item['link']
            print(f"처리 중: {url}")
            
            # 크롤링 실행 (리스트 반환)
            raw_contents = get_naver_blog_content(url)
            
            # 데이터 정제 및 청크 분할
            for content in raw_contents:
                cleaned = clean_text(content)  # 텍스트 정제
                chunks = chunk_text(cleaned)  # 청크 분할
                
                # 임베딩 생성 및 저장
                for chunk in chunks:
                    embedding = get_embedding(chunk)
                    index.add(np.array([embedding]))
                    metadata_store.append({
                        "text": chunk,
                        "url": url,
                        "title": item['title']
                    })
        print(f"총 {len(metadata_store)}개의 청크가 처리되었습니다.")
    
        # 사용자 질문 처리 및 응답 생성 예시
        query = f"{place_name}에 대해 긍정/부정 %로 나타내줘"
        openai_message(query)
    else:
        print("API 요청 실패 또는 결과 없음")

육림객잔
처리 중: https://blog.naver.com/k2jongin/223560846137
<<본문 크롤링이 완료되었습니다.>>
처리 중: https://blog.naver.com/mistuki07/223159897071
<<본문 크롤링이 완료되었습니다.>>
처리 중: https://blog.naver.com/hallo--ween/223184620636
<<본문 크롤링이 완료되었습니다.>>
처리 중: https://blog.naver.com/l22430/223441073644
<<본문 크롤링이 완료되었습니다.>>
처리 중: https://blog.naver.com/izstar/223162706881
<<본문 크롤링이 완료되었습니다.>>
처리 중: https://blog.naver.com/noksooo1/223152261173
<<본문 크롤링이 완료되었습니다.>>
처리 중: https://blog.naver.com/musongsong/223447212525
<<본문 크롤링이 완료되었습니다.>>
처리 중: https://blog.naver.com/gil_yang93/223184095215
<<본문 크롤링이 완료되었습니다.>>
처리 중: https://blog.naver.com/skycon3/223360056030
<<본문 크롤링이 완료되었습니다.>>
처리 중: https://blog.naver.com/pikazuu/223266761845
<<본문 크롤링이 완료되었습니다.>>
총 23개의 청크가 처리되었습니다.
긍정: 90%  
부정: 10%
진미닭갈비 본점
처리 중: https://blog.naver.com/98cute/223724886986
<<본문 크롤링이 완료되었습니다.>>
처리 중: https://blog.naver.com/boot_dal/223614285550
<<본문 크롤링이 완료되었습니다.>>
처리 중: https://blog.naver.com/92186636/223558467038
<<본문 크롤링이 완료되었습니다.>>
처리 중: https:

Exception ignored in: <function Service.__del__ at 0x00000169D807CE50>
Traceback (most recent call last):
  File "C:\Users\kss87\miniconda3\envs\test_env\lib\site-packages\selenium\webdriver\common\service.py", line 200, in __del__
    self.stop()
  File "C:\Users\kss87\miniconda3\envs\test_env\lib\site-packages\selenium\webdriver\common\service.py", line 157, in stop
    self.send_remote_shutdown_command()
  File "C:\Users\kss87\miniconda3\envs\test_env\lib\site-packages\selenium\webdriver\common\service.py", line 137, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "C:\Users\kss87\miniconda3\envs\test_env\lib\urllib\request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
  File "C:\Users\kss87\miniconda3\envs\test_env\lib\urllib\request.py", line 519, in open
    response = self._open(req, data)
  File "C:\Users\kss87\miniconda3\envs\test_env\lib\urllib\request.py", line 536, in _open
    result = self._call_chain(self.h

처리 중: https://blog.naver.com/rmsahd/223313890409


In [43]:
# # 강원대학교 후문의 경도와 위도
# longitude = "127.73983"  # 경도
# latitude = "37.87760"    # 위도
# category_code = "CE7"    # 카테고리 코드 (예: 카페)


# # 프롬프트 초기화
# prompt = """지금부터 당신은 장소 리뷰어가 된다. 
# 아래 스팟 리스트에 대해서 google에 해당 스팟의 리뷰에 대해, 부가적인 내용없이 각 스팟에 대해 출력 형식에 맞춰 답변한다.

# 스팟 리스트
# """

# prompt2 = """
# 출력 형식
# 장소 이름/긍정평가:n%/부정평가:n%
# 출처: url
# """

# # API 호출 및 데이터 처리
# result = search_by_category(longitude, latitude, category_code)

# if result is not None:  # API 요청 성공 여부 확인
#     documents = result.get('documents', [])
#     if documents:  # 검색 결과가 있을 경우
#         for idx, place in enumerate(documents, start=1):
#             name = place.get('place_name')       # 장소 이름
#             address = place.get('address_name')  # 주소
            
#             prompt += str(idx) + ". " + name + ", " + address + "\n"
#     else:  # 검색 결과가 없을 경우
#         prompt += "검색 결과가 없습니다.\n"
# else:
#     print("API 요청에 실패했습니다.")

# # 프롬프트 완성
# prompt += prompt2

# print(prompt)